In [1]:
import json
import pathlib

from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

/home/aaron/Documents/github/ipa-gpt-interpret/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
database_path = pathlib.Path('references') / "Features.xlsx"
output_path = pathlib.Path('data') / 'mappings.json'

dataset_names = [
    'iggy12345/ru-reviews-classification-ipa',
    'iggy12345/allegro-reviews-ipa'
]

# Dataset Analysis
Because there are multiple unicode representations for IPA symbols, I want to find specifically the ones that we'll work with.

In [3]:
datasets = list(map(load_dataset, dataset_names))

In [4]:
characters = set()

def collect_characters(ds, split: str):
    for row in tqdm(ds[split]):
        for ci, c in enumerate(row['text-phoneme']):
            characters.add(c)

for dataset in datasets:
    collect_characters(dataset, 'train')
    collect_characters(dataset, 'validation')

print(characters)


100%|██████████| 1002/1002 [00:00<00:00, 20583.17it/s]

{'«', 'L', 'р', 'ä', 'Ó', 'Y', 'у', '¡', '★', 'ɑ', 'J', '7', '－', '。', 'Р', 'Т', '⭐', '”', '°', 'b', 'з', 'д', 'ʑ', 'ó', '–', '❄', '|', '4', 'ɡ', 'U', 'т', 'ɹ', 'Ю', 'á', 'p', '#', '`', 'ɨ', '&', 'f', 'T', '1', '➖', '*', '\n', 'Щ', 'К', '✨', '≈', '̃', 'i', 'W', 'ɭ', 'N', 'и', 'ъ', 'P', 't', '⚘', '₽', 'х', 'ə', 'б', 'z', 'ŋ', 'ś', '•', '»', 'З', '◡', 'Ш', '’', '×', 'г', 'Б', 'l', 'M', ';', 'ʂ', 'Н', 'F', '—', '｀', 'о', 'V', '„', 'н', 'е', 'B', '{', 'ф', '（', 'ë', '}', 'ö', 'В', '➕', 'ş', 'Й', 'ж', 'э', 'Ç', 'Г', '´', '♥', 'ʌ', '_', 'щ', '©', 'ы', '5', 'j', 'в', 'М', '✔', '⛔', 'r', 'Е', 'Q', "'", '@', '❣', 'ː', '❌', 'I', '✩', '？', 'Х', '~', 'n', '2', 'C', '‘', 'ć', 'ʲ', 'ɵ', 'm', 'ь', 'Я', 'ヽ', '[', 'g', 'x', '\\', 'а', 'k', 'd', '❤', 'v', 'ˈ', 'ɔ', 'А', 'я', '％', 'ł', '‼', 'e', 'И', '6', '☝', 'O', 'ɣ', 'y', 'ą', 'R', '-', '∀', 'S', 'ш', 'H', ']', 'X', 'Ы', 'ę', '.', '́', 'ı', '☹', '$', '"', '№', 'Z', 's', 'ʒ', 'О', '…', 'ω', 'Ф', 'Ł', 'л', '“', 'A', 'с', '，', 'Ч', 'ú', 'h', 'ɕ', 'Ц', 'ʃ

In [5]:
df = pd.read_excel(database_path)
print(df.head())

  Unnamed: 0 syllabic stress long consonantal sonorant continuant  \
0          ɒ        +      -    -           -        +          +   
1          ɑ        +      -    -           -        +          +   
2          ɶ        +      -    -           -        +          +   
3          a        +      -    -           -        +          +   
4          æ        +      -    -           -        +          +   

  delayed release approximant tap  ... anterior distributed strident lateral  \
0               0           +   -  ...        0           0        0       -   
1               0           +   -  ...        0           0        0       -   
2               0           +   -  ...        0           0        0       -   
3               0           +   -  ...        0           0        0       -   
4               0           +   -  ...        0           0        0       -   

  DORSAL high low front back tense  
0      +    -   +     -    +     0  
1      +    -   +     -    +  

In [6]:
list(df.columns[1:])

['syllabic',
 'stress',
 'long',
 'consonantal',
 'sonorant',
 'continuant',
 'delayed release',
 'approximant',
 'tap',
 'trill',
 'nasal',
 'voice',
 'spread gl',
 'constr gl',
 'LABIAL',
 'round',
 'labiodental',
 'CORONAL',
 'anterior',
 'distributed',
 'strident',
 'lateral',
 'DORSAL',
 'high',
 'low',
 'front',
 'back',
 'tense']

In [7]:
result = {
    'mappings': {},
    'features': {}
}

In [8]:
for ci, col in enumerate(list(df.columns)[1:]):
    result['features'][col] = ci

In [9]:
found = []
not_found = []
for ri, row in df.iterrows():
    features = []
    symbol = row['Unnamed: 0']
    symbol = symbol.replace('͡', '')
    if symbol not in characters:
        not_found.append(symbol)
    else:
        found.append(symbol)
    for col in result['features'].keys():
        feat = str(row[col])
        if feat == '+':
            features.append(result['features'][col])
        elif feat == '0':
            features.append(-result['features'][col])
    result['mappings'][symbol] = features

In [10]:
print(found)

['ɑ', 'a', 'ʌ', 'ɔ', 'o', 'ə', 'e', 'ɛ', 'ɵ', 'u', 'ɨ', 'y', 'i', 'ɪ', 'ɲ', 'ŋ', 'ɭ', 'r', 'n', 'm', 'l', 'q', 'ɕ', 'c', 'ç', 'ɣ', 'x', 'k', 'ɡ', 'ʑ', 'ʂ', 'ʒ', 'z', 'v', 't', 'ʃ', 's', 'p', 'f', 'd', 'b', 'w', 'j', 'ɹ', 'h']


In [11]:
print(not_found)

['ɒ', 'ɶ', 'æ', 'ɤ', 'ɘ', 'œ', 'ɞ', 'ø', 'ɯ', 'ʊ', 'ʉ', 'ʏ', 'ŋ+', 'ʟ', 'ɫ', 'ɴ', 'ʀ', 'ʎ', 'ŋ˗', 'ʟ', 'ʟ̠', 'ɳ', 'ʙ', 'ɺ', 'ɻ', 'ɽ', 'ɾ', 'ɱ', 'ʔ', 'ɣ+', 'x+', 'k+', 'ɡ+', 'k+x+', 'ɡ+ɣ+', 'ħ', 'ʕ', 'ʁ', 'χ', 'ɢ', 'ɉ', 'ʝ', 'dʑ', 'tɕ', 'ɣ̠ ', 'x̠', 'k̠', 'ɡ̠', 'ʈ', 'ɖ', 'ɬ', 'ʐ', 'ɸ', 'θ', 'ɮ', 'ð', 'β', 'dʒ', 'dz', 'dɮ', 'd̠ɮ̠', 'tʃ', 't̠ɬ̠', 'ts', 'tɬ', 't̪s̪', 't̪ɬ̪', 'd̪z̪', 'd̪ɮ̪', 'ʈʂ', 'ɖʐ', 'pf', 'bv', 'pɸ', 'bβ', 't̪θ', 'd̪ð', 'cç', 'ɉʝ', 'kx', 'k̠x̠', 'ɡɣ', 'ɡ̠̠ɣ̠', 'qχ', 'ɢʁ', 'ɧ', 'kp', 'gb', 'pt', 'bd', 'ɰ', 'ɰ̠', 'ɥ', 'ʋ', 'ʍ', 'ɦ']


In [12]:
available_symbols = set(found + not_found)
not_fulfilled = [c for c in characters if c not in available_symbols]
print(not_fulfilled)

['«', 'L', 'р', 'ä', 'Ó', 'Y', 'у', '¡', '★', 'J', '7', '－', '。', 'Р', 'Т', '⭐', '”', '°', 'з', 'д', 'ó', '–', '❄', '|', '4', 'U', 'т', 'Ю', 'á', '#', '`', '&', 'T', '1', '➖', '*', '\n', 'Щ', 'К', '✨', '≈', '̃', 'W', 'N', 'и', 'ъ', 'P', '⚘', '₽', 'х', 'б', 'ś', '•', '»', 'З', '◡', 'Ш', '’', '×', 'г', 'Б', 'M', ';', 'Н', 'F', '—', '｀', 'о', 'V', '„', 'н', 'е', 'B', '{', 'ф', '（', 'ë', '}', 'ö', 'В', '➕', 'ş', 'Й', 'ж', 'э', 'Ç', 'Г', '´', '♥', '_', 'щ', '©', 'ы', '5', 'в', 'М', '✔', '⛔', 'Е', 'Q', "'", '@', '❣', 'ː', '❌', 'I', '✩', '？', 'Х', '~', '2', 'C', '‘', 'ć', 'ʲ', 'ь', 'Я', 'ヽ', '[', 'g', '\\', 'а', '❤', 'ˈ', 'А', 'я', '％', 'ł', '‼', 'И', '6', '☝', 'O', 'ą', 'R', '-', '∀', 'S', 'ш', 'H', ']', 'X', 'Ы', 'ę', '.', '́', 'ı', '☹', '$', '"', '№', 'Z', 'О', '…', 'ω', 'Ф', 'Ł', 'л', '“', 'A', 'с', '，', 'Ч', 'ú', 'Ц', '8', 'ч', '✌', 'D', '/', 'ц', 'ˌ', '<', '?', '!', 'ß', 'У', ' ', 'Ь', 'K', 'к', '>', '☺', 'é', 'ñ', '℅', '❗', 'ё', 'м', 'ﾉ', '9', 'ю', '=', 'Л', '÷', ':', 'Д', '）', '♡', '！

We'll have to do some fancy parsing to handle diphthongs, but it looks like the parsing is working correctly and the unicode used for the characters matches the unicode in the datasets themselves.

In [14]:
with open(output_path, 'w+') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)